In [1]:
import pathpyG as pp
from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index
import torch

from collections import deque

from scipy.sparse.csgraph import bellman_ford, dijkstra
import numpy as np
from time import time
from collections import defaultdict


from tqdm import tqdm

In [2]:
t_sp = pp.TemporalGraph.from_csv('../data/sociopatterns_highschool_2013_train.tedges')
print(t_sp)
print(torch.unique(t_sp.data.t).size(0))

Temporal Graph with 327 nodes, 8950 unique edges and 220378 events in [1385982080.0, 1386163840.0]

Graph attributes
	t		<class 'torch.Tensor'> -> torch.Size([220378])
	src		<class 'torch.Tensor'> -> torch.Size([220378])
	dst		<class 'torch.Tensor'> -> torch.Size([220378])

579


/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'t', 'src', 'dst'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [3]:
m = pp.MultiOrderModel.from_temporal_graph(t_sp, delta=3600, max_order=2)
print(m)

MultiOrderModel with max. order 2


In [4]:
t_ants = pp.TemporalGraph.from_csv('../data/ants_2_2_val.tedges')
print(t_ants)

Temporal Graph with 68 nodes, 506 unique edges and 1045 events in [899.0, 1796.0]

Graph attributes
	t		<class 'torch.Tensor'> -> torch.Size([1045])
	src		<class 'torch.Tensor'> -> torch.Size([1045])
	dst		<class 'torch.Tensor'> -> torch.Size([1045])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'t', 'src', 'dst'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [7]:
m = pp.MultiOrderModel.from_temporal_graph(t_ants, delta=30, max_order=10)
print(m)
print(m.layers[1].data.edge_weight)

MultiOrderModel with max. order 10
tensor([ 2.,  1.,  1.,  3.,  3.,  1.,  2.,  1.,  1.,  1.,  1.,  2.,  1.,  1.,
         1.,  1.,  1.,  1.,  4.,  1.,  1.,  1.,  1.,  5.,  2.,  6.,  6.,  1.,
         1.,  3.,  3.,  1.,  1.,  3.,  1.,  2.,  9.,  3.,  1.,  1.,  4.,  1.,
         1.,  1.,  4.,  2.,  3.,  2.,  1.,  4.,  3.,  2.,  4.,  1.,  1.,  3.,
         1.,  2.,  3.,  1.,  1.,  4.,  2.,  9.,  5.,  1.,  2.,  4.,  1.,  2.,
         1.,  1.,  1.,  3.,  4.,  1.,  2.,  3.,  4.,  1.,  1.,  3.,  4.,  4.,
         3.,  4.,  3.,  1.,  2.,  3.,  1.,  2.,  4.,  2.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  2.,  1.,  3.,  1.,  4.,  1.,  1.,  1.,  1.,  5.,  2.,
         1.,  1.,  2.,  1.,  1.,  2.,  1.,  4.,  3.,  4.,  1.,  2.,  1.,  1.,
         1.,  2.,  3.,  1.,  4.,  1.,  3.,  3.,  1.,  1.,  3.,  1.,  2.,  1.,
         1.,  4.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  3.,  1.,  3.,  1.,  1.,
         1.,  1.,  3.,  2.,  2.,  2.,  2.,  1.,  1.,  2.,  4.,  3.,  1.,  2.,
         2.,  3.,  5.,  5.,  

In [3]:
bw = pp.algorithms.centrality.temporal_betweenness_centrality(t_sp, delta=3600)
print(bw)

 31%|███       | 102/327 [26:29<58:26, 15.59s/it]  


KeyboardInterrupt: 

In [4]:
bw = pp.algorithms.centrality.temporal_betweenness_centrality(t_ants, delta=30)
print(bw)

100%|██████████| 594/594 [00:00<00:00, 4581.51it/s]


defaultdict(<function temporal_betweenness_centrality.<locals>.<lambda> at 0x7f9453f36cb0>, {0: 9.083333333333336, 7: 15.0, 10: 114.12687232787852, 1: 5.0, 6: 9.5, 12: 35.060389610389606, 22: 58.61201533244884, 49: 2.500000000000001, 2: 346.2515994755158, 27: 249.9532851737187, 4: 140.7880952380952, 42: 28.220839755359876, 28: 146.0366185070518, 65: 15.791666666666675, 29: 190.04444444444454, 24: 26.736796536796536, 5: 126.14722222222221, 3: 40.32903828197944, 35: 3.9999999999999996, 9: 7.5, 17: 78.59657287157289, 20: 68.21558441558443, 48: 27.916666666666664, 11: 99.25000000000001, 15: 53.07553688141922, 8: 5.8571428571428585, 26: 118.34098235785545, 37: -0.33333333333333304, 34: 58.120919946926136, 23: 49.347619047619034, 16: -0.1666666666666714, 32: 1.3333333333333286, 46: 2.40126050420168, 19: -1.8611111111111098, 36: 1.0000000000000004, 39: 2.0, 14: 25.33333333333333, 31: 1.5, 21: 0.5555555555555536, 33: 2.1666666666666705, 13: 0.0, 18: -5.19444444444445, 25: -1.9984014443252818e-

In [7]:
tedges = [('a', 'b', 1), ('b', 'c', 5), ('c', 'd', 9), ('c', 'e', 9),
              ('c', 'f', 11), ('f', 'a', 13), ('a', 'g', 18), ('b', 'f', 21),
              ('a', 'g', 26), ('c', 'f', 27), ('h', 'f', 27), ('g', 'h', 28),
              ('a', 'c', 30), ('a', 'b', 31), ('c', 'h', 32), ('f', 'h', 33),
              ('b', 'i', 42), ('i', 'b', 42), ('c', 'i', 47), ('h', 'i', 50)]
t_long = pp.TemporalGraph.from_edge_list(tedges)
c = pp.algorithms.centrality.temporal_closeness_centrality(t_long, 5)
print(c)

100%|██████████| 17/17 [00:00<00:00, 4453.94it/s]

Created temporal event DAG with 38 nodes and 47 edges
{'a': 12.0, 'b': 16.0, 'c': 16.0, 'd': 14.666666666666666, 'e': 14.666666666666666, 'f': 24.0, 'g': 14.666666666666666, 'h': 28.0, 'i': 24.0}


In [9]:
c = pp.algorithms.centrality.temporal_closeness_centrality(t_sp, 3600)

  0%|          | 0/1157 [00:00<?, ?it/s]

  4%|▎         | 43/1157 [00:05<02:18,  8.05it/s]


KeyboardInterrupt: 

In [4]:
print(c)

{'454': 41671.338095238054, '640': 42220.10476190471, '1': 31384.485714285773, '939': 43385.1666666666, '185': 46485.271428571396, '258': 42662.533333333326, '55': 41131.62698412695, '170': 43822.54999999994, '9': 55537.204761904715, '453': 42306.650000000016, '45': 54390.77142857138, '14': 42915.57142857141, '190': 51695.83809523803, '400': 37502.9365079365, '637': 40505.24126984126, '255': 43516.342857142794, '275': 53779.13333333323, '176': 52638.13333333328, '533': 36436.70952380954, '116': 46335.46666666662, '151': 46517.87142857134, '866': 51019.77619047604, '280': 40957.63095238092, '484': 38709.39523809525, '243': 42567.83809523808, '687': 43355.67142857139, '54': 44738.06666666662, '364': 43477.92142857135, '374': 41239.258730158705, '295': 37942.38968253968, '441': 44693.04761904753, '101': 46261.46984126977, '425': 40808.355411255354, '47': 34439.18333333336, '241': 40734.0880952381, '179': 51910.84285714278, '202': 47549.81666666654, '63': 48038.816666666564, '564': 38769.9

In [5]:
def temporal_event_graph(g: pp.TemporalGraph, delta=1): 

    print(g.data.edge_index)

    # generate temporal event DAG
    edge_index = pp.algorithms.lift_order_temporal(g, delta)
    print(edge_index)

    # Add indices of first-order nodes as src and dst of paths in augmented
    # temporal event DAG
    src_edges_src = g.data.edge_index[0] + g.data.edge_index.size(1)
    print(src_edges_src)
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))

    dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    dst_edges_dst = g.data.edge_index[1] + g.data.edge_index.size(1) + g.N
    print(dst_edges_dst)

    # add edges from source to edges and from edges to destinations
    src_edges = torch.stack([src_edges_src, src_edges_dst])
    dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges, dst_edges], dim=1)

    # create sparse scipy matrix
    event_graph = pp.Graph.from_edge_index(edge_index) 
    return event_graph

In [10]:
eg = pp.algorithms.lift_order_temporal(t_long, delta=5)
print(eg)

100%|██████████| 17/17 [00:00<00:00, 3068.65it/s]

tensor([[ 0,  1,  1,  4,  5,  8, 12],
        [ 1,  2,  3,  5,  6, 11, 14]])


In [11]:
print(temporal_event_graph(t_long, delta=5))

tensor([[0, 1, 2, 2, 2, 5, 0, 1, 0, 7, 2, 6, 0, 0, 2, 5, 1, 8, 2, 7],
        [1, 2, 3, 4, 5, 0, 6, 5, 6, 5, 5, 7, 2, 1, 7, 7, 8, 1, 8, 8]])


100%|██████████| 17/17 [00:00<00:00, 3405.44it/s]

tensor([[ 0,  1,  1,  4,  5,  8, 12],
        [ 1,  2,  3,  5,  6, 11, 14]])
tensor([20, 21, 22, 22, 22, 25, 20, 21, 20, 27, 22, 26, 20, 20, 22, 25, 21, 28,
        22, 27])
tensor([30, 31, 32, 33, 34, 29, 35, 34, 35, 34, 34, 36, 31, 30, 36, 36, 37, 30,
        37, 37])
Directed graph with 38 nodes and 47 edges

Graph attributes
	num_nodes		<class 'int'>



In [13]:
def betweenness_brandes(g: pp.Graph, sources = None):
    bw = defaultdict(lambda: 0.0)

    if sources == None:
        sources = [v for v in g.nodes]

    for s in tqdm(sources):
        S = list()
        P = defaultdict(list)

        sigma = defaultdict(lambda: 0)  
        sigma[s] = 1

        d = defaultdict(lambda: -1)        
        d[s] = 0

        Q = [s]
        while Q:
            v = Q.pop(0)
            S.append(v)
            for w in g.successors(v):
                if d[w] < 0:
                    Q.append(w)
                    d[w] = d[v] + 1
                if d[w] == d[v] + 1:
                    # we found shortest path from s via v to w
                    sigma[w] = sigma[w] + sigma[v]
                    P[w].append(v)
        delta = defaultdict(lambda: 0.0)
        while S:
            w = S.pop()
            for v in P[w]:
                delta[v] = delta[v] + sigma[v]/sigma[w] * (1 + delta[w])
                if v != w:
                    bw[w] = bw[w] + delta[w]
    return bw

In [14]:
g = pp.Graph.from_edge_list([('a', 'c'), ('b', 'c'), ('c', 'd'), ('c', 'e')])
betweenness_brandes(g, g.nodes)

5it [00:00, 3583.04it/s]


defaultdict(<function __main__.betweenness_brandes.<locals>.<lambda>()>,
            {'e': 0.0, 'd': 0.0, 'c': 4.0})

In [15]:
def temporal_event_graph(g: pp.TemporalGraph, delta = 1):
    # generate temporal event DAG
    edge_index = pp.algorithms.lift_order_temporal(g, delta)    

    # Add indices of first-order nodes as src and dst of paths in augmented
    # temporal event DAG
    print(g.data.edge_index)
    edges = [f'{v}-{w}-{t}' for v, w, t in g.temporal_edges]
    print(edges)
    src_edges_src = g.data.edge_index[0] + g.M
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))

    src = [f'{v}-src' for v in g.nodes]
    tgt = [f'{v}-tgt' for v in g.nodes]

    dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    dst_edges_dst = g.data.edge_index[1] + g.M + g.N

    # add edges from source to edges and from edges to destinations
    src_edges = torch.stack([src_edges_src, src_edges_dst])
    dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges, dst_edges], dim=1)

    # create sparse scipy matrix
    mapping = pp.IndexMap(edges + src + tgt)
    event_graph = pp.Graph.from_edge_index(edge_index, mapping) 
    return event_graph

In [25]:
# def fo_node(v, g, src_indices) -> int:
#     # if v is one of the source nodes, return corresponding first-order node

    
# def fo_src(v, g, src_indices):
#     if v in src_indices:
#         return v - g.M
#     else:
#         return g.data.edge_index[0,v].item()

def temporal_betweenness_brandes(g: pp.TemporalGraph, delta=1):

    print(g.data.edge_index)

    # generate temporal event DAG
    edge_index = pp.algorithms.lift_order_temporal(g, delta)

    # Add indices of first-order nodes as src and dst of paths in augmented
    # temporal event DAG
 #   print(g.data.edge_index)
    #edges = [f'{v}-{w}-{t}' for v, w, t in g.temporal_edges]
#    print(edges)
    src_edges_src = g.data.edge_index[0] + g.M
    src_edges_dst = torch.arange(0, g.data.edge_index.size(1))

    #src = [f'{v}-src' for v in g.nodes]
    #tgt = [f'{v}-tgt' for v in g.nodes]

    # dst_edges_src = torch.arange(0, g.data.edge_index.size(1))
    # dst_edges_dst = g.data.edge_index[1] + g.M + g.N

    # add edges from source to edges and from edges to destinations
    src_edges = torch.stack([src_edges_src, src_edges_dst])
    # dst_edges = torch.stack([dst_edges_src, dst_edges_dst])
    edge_index = torch.cat([edge_index, src_edges], dim=1)

    # create sparse scipy matrix
    #mapping = pp.IndexMap(edges + src + tgt)
    event_graph = pp.Graph.from_edge_index(edge_index, num_nodes=g.M+g.N)
    print(event_graph)
    #pp.plot(event_graph, node_label=[i for i in event_graph.nodes])
    #print(edge_index)

    # # sources = first-order source nodes in temporal event graph
    src_indices = set(torch.unique(src_edges_src).tolist())
    #print(src_edges_src-g.M)
    # tgt_indices = set(torch.unique(dst_edges_dst).tolist())
    #print(src_indices)
    # print(tgt_indices)

    e_i = g.data.edge_index.numpy()

    fo_nodes = dict()
    for v in event_graph.nodes:
        if v in src_indices:
            fo_nodes[v] = v - g.M
        else: # return first-order target node otherwise
            fo_nodes[v] = e_i[1,v]

    # start from indegree zero nodes
    #roots = torch.where((degree(g.data.edge_index[1])==0))[0]
    #dist, _ = pp.algorithms.temporal_shortest_paths(g, delta=delta)
    #print(dist)
    bw = defaultdict(lambda: 0.0)

    # for all first-order nodes
    for s in tqdm(src_indices):
        t_start = time()
        print('source', g.mapping.to_id(fo_nodes[s]))

        # for any given s, d[v] is the shortest path distance from s to v
        # Note that here we calculate topological distances from sources to events (i.e. time-stamped edges)
        delta_ = defaultdict(lambda: 0.0)

        # for any given s, sigma[v] counts shortest paths from s to v
        sigma = defaultdict(lambda: 0.0)          
        sigma[s] = 1

        sigma_fo = defaultdict(lambda: 0.0)       
        sigma_fo[fo_nodes[s]] = 1

        dist = defaultdict(lambda: -1)
        dist[s] = 0

        dist_fo = defaultdict(lambda: -1)
        dist_fo[fo_nodes[s]] = 0
                
        # for any given s, P[v] is the set of predecessors of v on shortest paths from s
        P = defaultdict(list)

        # Q is a queue, so we append at the end and pop from the start
        Q = deque()  
        Q.append(s)

        # S is a stack, so we append at the end and pop from the end
        S = list()
    
        # dijkstra with path counting
        while Q:
            v = Q.popleft()
            #print('popped ', v)
            # for all successor events within delta
            for w in event_graph.successors(v):

                # we dicover w for the first time
                if dist[w] == -1:
                    dist[w] = dist[v] + 1
                    if dist_fo[fo_nodes[w]] == -1:
                        dist_fo[fo_nodes[w]] = dist[v] + 1
                    S.append(w)
                    Q.append(w)
                # we found a shortest path to event w via event v
                if dist[w] == dist[v] + 1:
                    sigma[w] += sigma[w] + sigma[v]
                    P[w].append(v)
                    # we found a shortest path to first-order node of event w
                    if dist[w] == dist_fo[fo_nodes[w]]:
                        sigma_fo[fo_nodes[w]] += sigma[v]
        #print('S =', S)
        #print('P', P)
        #print('d', dist)
        print('finished BFS ', (time()- t_start))
        c = 0
        for i in dist_fo:
            if dist_fo[i] >=0:
                c+= 1
        bw[fo_nodes[s]] = bw[fo_nodes[s]] - c + 1
        #print(bw[fo_node(s, g, src_indices)])

        # We computed top. shortest path distances and shortest path counts from (first-order) source nodes to all temporal events
        # we must now project this to the first-order target nodes of those events
        while S:
            w = S.pop()
            # work backwards through paths to all targets and sum delta and sigma

            # check whether shortest path from s to event w is also shortest path to first-order target of w
            # if d[w] == d_fo[w_fo]:            
            if dist[w] == dist_fo[fo_nodes[w]]:
                # v_fo = fo_tgt(v, g, src_indices, tgt_indices)
                delta_[w] += (sigma[w]/sigma_fo[fo_nodes[w]])
            for v in P[w]:
                delta_[v] += (sigma[v]/sigma[w]) * delta_[w]
                bw[fo_nodes[v]] += delta_[w] * (sigma[v]/sigma[w])
    bw_id = defaultdict(lambda: 0.0)
    for idx in bw:
        bw_id[g.mapping.to_id(idx)] = bw[idx]
    return bw_id


In [26]:
temporal_betweenness_brandes(t_sp, delta=3600)

tensor([[ 43,  43,  44,  ..., 202, 162,  76],
        [ 45,  44,  43,  ..., 262,  76, 162]])


  0%|          | 0/579 [00:00<?, ?it/s]

100%|██████████| 579/579 [00:39<00:00, 14.52it/s]


Directed graph with 220705 nodes and 18249204 edges

Graph attributes
	num_nodes		<class 'int'>



  0%|          | 0/327 [00:00<?, ?it/s]

...

source 1
finished BFS  9.732733726501465


  1%|          | 3/327 [00:56<1:32:16, 17.09s/it]

source 939


  1%|          | 3/327 [01:19<2:22:30, 26.39s/it]


KeyboardInterrupt: 

In [61]:
# Example with two shortest time-respecting paths from a to e via c or d
t = pp.TemporalGraph.from_edge_list([('a', 'b', 1), ('b', 'c', 2), ('b', 'd', 2), ('c', 'e', 3), ('d', 'e', 3)])
print(t.data.edge_index)
print(t.mapping)

bw_1 = temporal_betweenness_brandes(t, delta=1)

for v in t.nodes:
    print(v, bw_1[t.mapping.to_idx(v)])

tensor([[0, 1, 1, 2, 3],
        [1, 2, 3, 4, 4]])
a -> 0
b -> 1
c -> 2
d -> 3
e -> 4

tensor([[0, 1, 1, 2, 3],
        [1, 2, 3, 4, 4]])


100%|██████████| 4/4 [00:00<00:00, 993.62it/s]

source d
source a
source b
source c
a 0.0
b 3.0
c 1.0
d 1.0
e 0.0


In [62]:
bw = temporal_betweenness_brandes(t_long, 5)
for v in t_long.nodes:
    print(v, bw[t_long.mapping.to_idx(v)])

tensor([[0, 1, 2, 2, 2, 5, 0, 1, 0, 7, 2, 6, 0, 0, 2, 5, 1, 8, 2, 7],
        [1, 2, 3, 4, 5, 0, 6, 5, 6, 5, 5, 7, 2, 1, 7, 7, 8, 1, 8, 8]])


100%|██████████| 7/7 [00:00<00:00, 1022.18it/s]

source a
source b
source c
source f
source g
source h
source i
a 2.0
b 2.0
c 4.5
d 0.0
e 0.0
f 2.0
g 0.5
h 0.0
i 0.0


In [32]:
t = pp.TemporalGraph.from_edge_list([('a', 'b', 1), ('b', 'c', 2), ('c', 'd', 3), ('b', 'd', 3)])
print(t.data.edge_index)
print(t.mapping)

bw_1 = temporal_betweenness_brandes(t, delta=1)
bw_2 = temporal_betweenness_brandes(t, delta=2)

for v in t.nodes:
    print(v, bw_1[t.mapping.to_idx(v)], bw_2[t.mapping.to_idx(v)])

tensor([[0, 1, 2, 1],
        [1, 2, 3, 3]])
a -> 0
b -> 1
c -> 2
d -> 3

tensor([[0, 1, 2, 1],
        [1, 2, 3, 3]])


100%|██████████| 3/3 [00:00<00:00, 3437.95it/s]


{4, 5, 6}


100%|██████████| 3/3 [00:00<00:00, 875.52it/s]


source a
source b
source c
tensor([[0, 1, 2, 1],
        [1, 2, 3, 3]])


100%|██████████| 3/3 [00:00<00:00, 2423.05it/s]


{4, 5, 6}


100%|██████████| 3/3 [00:00<00:00, 1125.28it/s]

source a
source b
source c
a 0.0 -1.0
b 1.0 1.0
c 1.0 0.0
d 0.0 0.0


In [93]:
pp.algorithms.temporal_shortest_paths(t, delta=2)

100%|██████████| 3/3 [00:00<00:00, 2388.10it/s]

Created temporal event DAG with 12 nodes and 11 edges


(array([[ 0.,  1.,  2.,  2.],
        [inf,  0.,  1.,  1.],
        [inf, inf,  0.,  1.],
        [inf, inf, inf,  0.]]),
 array([[-9999,     0,     1,     3],
        [-9999, -9999,     1,     3],
        [-9999, -9999, -9999,     2],
        [-9999, -9999, -9999, -9999]], dtype=int32))

In [57]:
pp.algorithms.temporal_shortest_paths(t, delta=1)

100%|██████████| 3/3 [00:00<00:00, 3233.02it/s]

Created temporal event DAG with 12 nodes and 10 edges


(array([[ 0.,  1.,  2.,  3.],
        [inf,  0.,  1.,  1.],
        [inf, inf,  0.,  1.],
        [inf, inf, inf,  0.]]),
 array([[-9999,     0,     1,     2],
        [-9999, -9999,     1,     3],
        [-9999, -9999, -9999,     2],
        [-9999, -9999, -9999, -9999]], dtype=int32))

In [313]:
t = pp.TemporalGraph.from_edge_list([('a', 'c', 1), ('c', 'd', 2), ('b', 'c', 3), ('c', 'e', 4)])
print(t.mapping)

temporal_betweenness_brandes(t, delta=1)
# 4 = a_src
# 5 = c_src
# 7 = b_src

# 10 = c_tgt
# 11 = d_tgt
# 13 = e_tgt

a -> 0
c -> 1
d -> 2
b -> 3
e -> 4

tensor([[0, 1, 3, 1],
        [1, 2, 1, 4]])


100%|██████████| 3/3 [00:00<00:00, 2406.37it/s]

10
[0]
11
[1]
13
[]
10
[]
11
[1]
13
[3]
10
[2]
11
[]
13
[3]


defaultdict(<function __main__.temporal_betweenness_brandes.<locals>.<lambda>()>,
            {1: 0.0, 2: 0.0, 4: 0.0})

In [253]:
t = pp.TemporalGraph.from_edge_list([('a', 'c', 1), ('c', 'd', 2), ('b', 'c', 3), ('c', 'e', 4),('a', 'c', 1), ('c', 'e', 2), ('b', 'c', 3), ('c', 'd', 4)])
print(t.mapping)

temporal_betweenness_brandes(t, delta=1)
# 4 = a_src
# 5 = c_src
# 7 = b_src

# 10 = c_tgt
# 11 = d_tgt
# 13 = e_tgt

a -> 0
c -> 1
d -> 2
b -> 3
e -> 4

tensor([[0, 0, 1, 1, 3, 3, 1, 1],
        [1, 1, 2, 4, 1, 1, 4, 2]])


100%|██████████| 4/4 [00:00<00:00, 2609.21it/s]


tensor([[0, 0, 1, 1, 4, 4, 5, 5],
        [2, 3, 2, 3, 6, 7, 6, 7]])
{8, 9, 11}
{17, 14, 15}


8it [00:00, 3661.15it/s]


defaultdict(<function __main__.temporal_betweenness_brandes.<locals>.<lambda>()>,
            {4: 0.0, 2: 0.0})

In [138]:
event_graph = temporal_event_graph(t, delta=1)
pp.plot(event_graph, node_label = [v for v in event_graph.nodes])

100%|██████████| 4/4 [00:00<00:00, 3337.42it/s]

tensor([[0, 1, 3, 1],
        [1, 2, 1, 4]])
['a-c-1.0', 'c-d-2.0', 'b-c-3.0', 'c-e-4.0']


In [179]:
betweenness_brandes(event_graph, sources=['a-src', 'b-src', 'c-src'])

100%|██████████| 3/3 [00:00<00:00, 2910.69it/s]


defaultdict(<function __main__.betweenness_brandes.<locals>.<lambda>()>,
            {'d-tgt': 0.0,
             'c-tgt': 0.0,
             'c-d-2.0': 2.0,
             'a-c-1.0': 3.0,
             'e-tgt': 0.0,
             'c-e-4.0': 2.0,
             'b-c-3.0': 3.0})

tensor([[0, 1, 3, 1],
        [1, 2, 1, 4]])


100%|██████████| 4/4 [00:00<00:00, 2712.57it/s]


tensor([[ 0,  2,  4,  5,  7,  5,  0,  1,  2,  3],
        [ 1,  3,  0,  1,  2,  3, 10, 11, 10, 13]])
{4, 5, 7}


  0%|          | 0/3 [00:00<?, ?it/s]


UnboundLocalError: local variable 'fo_src' referenced before assignment

In [75]:
temporal_betweenness_brandes(t_sp, delta=3600)

  8%|▊         | 25/327 [13:32<2:43:34, 32.50s/it]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3320/540718842.py", line 1, in <module>
    temporal_betweenness_brandes(t_sp, delta=3600)
  File "/tmp/ipykernel_3320/1211725645.py", line 36, in temporal_betweenness_brandes
    return betweenness_brandes(event_graph, src_indices)
  File "/tmp/ipykernel_3320/2494254184.py", line -1, in betweenness_brandes
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/opt/conda/lib/python3.10